In [3]:
from llama_index.readers.file import PagedCSVReader
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
# from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent, ReActAgent
from llama_index.core.workflow import Context

from shoe_recommender_engine import ShoeRecommenderEngine
from shoe_retrieval import ShoeRetrieval

In [64]:
file_path = 'data/2025 HOOP SHEET PROCESSED.csv'
llm_kwargs = {'model': 'gpt-4o-mini', 'request_timeout': 360.0, 'temperature': 0.1}

In [65]:
Settings.llm = OpenAI(**llm_kwargs)
Settings.embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v3", trust_remote_code=True)

In [66]:
documents = PagedCSVReader().load_data('data/2025 HOOP SHEET PROCESSED.csv')

In [67]:
index = VectorStoreIndex.from_documents(documents)

In [68]:
query_engine = index.as_query_engine(similarity_top_k = 1)
query_engine.query("How is Joker 1's cushioning?").response

'The cushioning of the Joker 1 is characterized by a midsole foam made of CO2 and thermoplastic, which contributes to its overall comfort and shock absorption. It has a bounce rating of 5.0 and a shock absorption rating of 4.5, indicating that it provides a good level of cushioning and support during use.'

In [69]:
retriever = index.as_retriever(similarity_top_k=1)
retriever.retrieve("How is Joker 1's cushioning?")[0].text

"NAME: 361 JOKER 1 GT\n30 SEC HEATING (F) (INTERNAL): 117.4\nBOUNCE HEIGHT AVERAGE (cm): 45.0\nHEEL HEIGHT (cm): 2.1\nHEEL-BALL DROP (cm): 0.5\nWEIGHT oz (US MEN'S 11): 15.2\nDEGREE OF SLIP: 44.0\nSHANK SCORE: 0.3\nSPEED RATIO: 3.26\nDUROMETER: 17.9\nLAST SHAPE (DEGREE OF INFLARE): 15\nLENGTH (cm): 28.8\nMETATARSAL WIDTH(cm): 10.0\nTOEBOX WIDTH(cm): 10.9\nTOE BOX TAPER (DEGREES): 24.0\nRESTING POSITION (DEGREES OF EVERSION): 1.0\nLOADED POSITION (DEGREES OF EVERSION): 7.0\nHEEL WIDTH (cm): 14.0\nNARROWEST MIDFOOT MEASUREMENT (cm): \nOVERALL RATING SCORE: 36.5\nMIDSOLE FOAM: CO2+THERMOPLASTIC\nSHANK MATERIAL: PLASTIC\nUPPER DURABILITY 0 = NOT THROUGH 1ST LAYER 1 = THROUGH 1ST LAYER: 0.0\nOUTSOLE DURABILITY: <1mm\nCONTAINMENT: 5.0\nBOUNCE: 5.0\nSHOCK ABSORPTION: 4.5\nSPEED: 4.5\nDURABILITY: 4.0\nCOMFORT: 4.5\nSUPPORT: 4.5\nPLAYABILITY: 4.5\nFITS ORTHOTICS: True\nFLAT ARCH: True\nNEUTRAL ARCH: True\nHIGH ARCH: True\nHARDWOOD SURFACE: True\nRUBBERIZED SURFACE: False\nASPHALT SURFACE: False

In [71]:
query_engine.query("Recommend me some lightweight shoes").response

"For lightweight options, consider shoes that weigh around 15.03 ounces (for US Men's size 11). Look for models that feature a combination of EVA and Zoom foam for cushioning while maintaining a lightweight feel. Additionally, ensure they have a good bounce height and support for various arch types, as well as compatibility with orthotics for added comfort."

In [73]:
retriever.retrieve("Recommend me some lightweight shoes")[0].text

"NAME: JORDAN WHY NOT ZER0.6\n30 SEC HEATING (F) (INTERNAL): 140.9\nBOUNCE HEIGHT AVERAGE (cm): 36.75\nHEEL HEIGHT (cm): 2.4\nHEEL-BALL DROP (cm): 0.6\nWEIGHT oz (US MEN'S 11): 15.03\nDEGREE OF SLIP: \nSHANK SCORE: \nSPEED RATIO: 2.45\nDUROMETER: 14.5\nLAST SHAPE (DEGREE OF INFLARE): 11\nLENGTH (cm): 28.7\nMETATARSAL WIDTH(cm): 9.2\nTOEBOX WIDTH(cm): \nTOE BOX TAPER (DEGREES): \nRESTING POSITION (DEGREES OF EVERSION): \nLOADED POSITION (DEGREES OF EVERSION): \nHEEL WIDTH (cm): \nNARROWEST MIDFOOT MEASUREMENT (cm): \nOVERALL RATING SCORE: 31.5\nMIDSOLE FOAM: EVA + ZOOM\nSHANK MATERIAL: PLASTIC\nUPPER DURABILITY 0 = NOT THROUGH 1ST LAYER 1 = THROUGH 1ST LAYER: 0.0\nOUTSOLE DURABILITY: 1.5 MM\nCONTAINMENT: \nBOUNCE: \nSHOCK ABSORPTION: \nSPEED: \nDURABILITY: \nCOMFORT: \nSUPPORT: \nPLAYABILITY: \nFITS ORTHOTICS: True\nFLAT ARCH: True\nNEUTRAL ARCH: True\nHIGH ARCH: True\nHARDWOOD SURFACE: True\nRUBBERIZED SURFACE: False\nASPHALT SURFACE: False\nFOREFOOT HEIGHT (cm): 1.7999999999999998"

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from shoe_retrieval import ShoeRetrieval

In [ ]:
retriever = ShoeRetrieval('data/2025 HOOP SHEET PROCESSED.csv')

In [42]:
retriever_tool = retriever.as_tool()

In [ ]:
agent = FunctionAgent(name = "shoe_retriever_agent",
                                    description = "An agent capable of retrieving specs given basketball shoe name(s) and discuss the performance of the shoe(s). "\
                                        "If multiple shoes are given, the agent should discuss the performance of each shoe one by one and compare them. This agent should be used **ONLY** when the shoe name(s) is given by the user.",
                                    system_prompt = "You are a helpful assistant that retrieves specs given basketball shoe name(s) and discuss the performance of the shoe(s) "\
                                        "If multiple shoes are given, the agent should discuss the performance of each shoe one by one and compare them. Use global_median_values as a reference of the average shoes' performance",
                                    tools = [retriever_tool],)

In [60]:
response = await agent.run("How is Joker 1's cushioning?")
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="The Joker 1 basketball shoe features a midsole made of CO2+Thermoplastic foam, which contributes to its cushioning performance. Here are some key metrics related to its cushioning:\n\n1. **Bounce Score**: The Joker 1 has a bounce score of 5.0 out of 5, indicating excellent responsiveness and energy return during play. This is above the global median value of 4.0, suggesting that the shoe provides a lively feel when jumping or sprinting.\n\n2. **Shock Absorption**: With a score of 4.5 out of 5, the shock absorption capability of the Joker 1 is also impressive. This means it effectively absorbs impact during landings, which can help reduce stress on the joints.\n\n3. **Comfort**: The comfort score is 4.5 out of 5, indicating that the shoe is designed to be comfortable for extended wear, which is crucial for basketball players who spend long hours on

In [61]:
message = response.response

In [62]:
message.content

"The Joker 1 basketball shoe features a midsole made of CO2+Thermoplastic foam, which contributes to its cushioning performance. Here are some key metrics related to its cushioning:\n\n1. **Bounce Score**: The Joker 1 has a bounce score of 5.0 out of 5, indicating excellent responsiveness and energy return during play. This is above the global median value of 4.0, suggesting that the shoe provides a lively feel when jumping or sprinting.\n\n2. **Shock Absorption**: With a score of 4.5 out of 5, the shock absorption capability of the Joker 1 is also impressive. This means it effectively absorbs impact during landings, which can help reduce stress on the joints.\n\n3. **Comfort**: The comfort score is 4.5 out of 5, indicating that the shoe is designed to be comfortable for extended wear, which is crucial for basketball players who spend long hours on the court.\n\n4. **Heel-Ball Drop**: The heel-ball drop is 0.5 cm, which is relatively low. This suggests that the shoe offers a closer-to-

In [76]:
from llama_index.core.prompts import PromptTemplate
from shoe_constants import COLUMN_DESCRIPTIONS

In [75]:
prompt_template = PromptTemplate(
            template="""
You are a shoe recommendation assistant.

Given a user query and the following column names from a shoe specification table:
{columns}

Determine:
1. Which column(s) should be **ranked** (sorted), and in what direction (asc/desc).
2. Which column(s) should be **filtered**, and the filter condition (e.g., < 150). **Only** filter columns if the query explicitly mentions them, you should not implicitly filter but sort.

Return a valid Python dictionary like this:
{{
  "ranking": [
    {{"column": "Weight oz/g", "sort": "asc"}},
    {{"column": "Cushioning Score", "sort": "desc"}}
  ],
  "filters": [
    {{"column": "Price", "op": "<", "value": 150}}
  ]
}}

and an explanation of your reasoning.
User query: "{query}"
"""
        )

In [77]:
prompt_template.format(columns = COLUMN_DESCRIPTIONS, query = "Lightweight basketball shoes")

'\nYou are a shoe recommendation assistant.\n\nGiven a user query and the following column names from a shoe specification table:\n{\'NAME\': \'brand + model name of the basketball shoe\', \'30 SEC HEATING (F) (INTERNAL)\': \'internal temperature after 30 seconds of heating, indicating breathability or ventilation\', \'BOUNCE HEIGHT AVERAGE (cm)\': \'average bounce height on the midsole\', \'HEEL HEIGHT (cm)\': \'height of the heel from the ground\', \'FOREFOOT HEIGHT (cm)\': \'height of the forefoot from the ground, lower indicating more court feeling\', \'HEEL-BALL DROP (cm)\': \'difference in height between heel and forefoot of the basketball shoe, higher indicating more achilles relief but less court feel\', "WEIGHT oz (US MEN\'S 11)": \'weight of the basketball shoe in ounces\', \'DEGREE OF SLIP\': \'angle of incline at which the basketball shoe begins to slip\', \'SHANK SCORE\': \'score based on the shank material and design for torsional stability\', \'SPEED RATIO\': \'ratio of 

In [17]:
llm_kwargs = {'model': 'gpt-4o-mini', 'request_timeout': 360.0, 'temperature': 0.1}
file_path = 'data/2025 HOOP SHEET PROCESSED.csv'
Settings.llm = OpenAI(**llm_kwargs)
Settings.embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v3", trust_remote_code=True)

shoe_recommender_tool = ShoeRecommenderEngine(file_path, llm_kwargs).as_tool()
shoe_retrieval_tool = ShoeRetrieval(file_path).as_tool()

shoe_recommender_agent = FunctionAgent(name = "shoe_recommender_agent",
                                description = "An agent capable of recommending basketball shoes based on user's preference using recommender tool.",
                                system_prompt = "You are a helpful assistant that recommends basketball shoes with clear explanationbased on user's preference using recommender tool. Handoff to shoe_retrieval_agent when specific shoe name(s) is given.",
                                tools = [shoe_recommender_tool],
                                can_handoff_to = ["shoe_retrieval_agent"]
                                )
shoe_retrieval_agent = FunctionAgent(name = "shoe_retrieval_agent",
                                description = "An agent capable of retrieving specs given basketball shoe name(s) and discuss the performance of the shoe(s). "\
                                    "If multiple shoes are given, the agent should discuss the performance of each shoe one by one and compare them. This agent should be used **ONLY** when the shoe name(s) is given by the user.",
                                system_prompt = "You are a helpful assistant that retrieves specs given basketball shoe name(s) and discuss the performance of the shoe(s) "\
                                    "If multiple shoes are given, the agent should discuss the performance of each shoe one by one and compare them. Use global_median_values as a reference of the average shoes' performance" \
                                    "Handoff to shoe_recommender_agent when the user asks for recommendations without specific shoe name(s).",
                                tools = [shoe_retrieval_tool],
                                can_handoff_to = ["shoe_recommender_agent"]
                                )
workflow = AgentWorkflow(agents = [shoe_recommender_agent, shoe_retrieval_agent], root_agent = 'shoe_recommender_agent',initial_state = {"recommendations": []})

context = Context(workflow)

In [18]:
response = await workflow.run("Compare adidas select 2.0 and 3.0", ctx = context)

In [19]:
response.response.content

'### Comparison of Adidas Select 2.0 and 3.0\n\n#### Adidas Select 2.0\n- **Weight**: 13.0 oz\n- **Heel Height**: 1.9 cm\n- **Bounce Height Average**: 29.5 cm\n- **Heel-Ball Drop**: 0.8 cm\n- **30 Sec Heating (F)**: 105.3°F\n- **Degree of Slip**: 43.0°\n- **Shank Score**: 0.3\n- **Speed Ratio**: 2.57\n- **Overall Rating Score**: 30.0/40\n- **Midsole Foam**: ETPU\n- **Containment**: 4.0/5\n- **Bounce**: 3.5/5\n- **Shock Absorption**: 3.5/5\n- **Speed**: 4.0/5\n- **Durability**: 3.5/5\n- **Comfort**: 4.0/5\n- **Support**: 3.5/5\n- **Playability**: 4.0/5\n- **Fits Orthotics**: Yes\n- **Arch Types Supported**: Flat, Neutral, High\n\n#### Adidas Select 3.0\n- **Weight**: 13.25 oz\n- **Heel Height**: 2.8 cm\n- **Bounce Height Average**: 31.5 cm\n- **Heel-Ball Drop**: 1.0 cm\n- **30 Sec Heating (F)**: 97.0°F\n- **Degree of Slip**: 43.0°\n- **Shank Score**: 0.4\n- **Speed Ratio**: 2.78\n- **Overall Rating Score**: 32.0/40\n- **Midsole Foam**: Not specified\n- **Containment**: 4.0/5\n- **Bounce

In [33]:
context.data['memory'].get('text')

/var/folders/5t/2k2m5syx3c53jbnddp_f19c00000gn/T/ipykernel_36846/1026804887.py:1: DeprecationWarning: `data` is deprecated, please use the `get` and `set` method to store data into the Context.
  context.data['memory'].get('text')


[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="Current state:\n{'recommendations': []}\n\nCurrent message:\nCompare adidas select 2.0 and 3.0\n")]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [ChoiceDeltaToolCall(index=0, id='call_Bntx2VFnOBcr0rd26XyG3kTI', function=ChoiceDeltaToolCallFunction(arguments='{"to_agent":"shoe_retrieval_agent","reason":"User has provided specific shoe names for comparison."}', name='handoff'), type='function')]}, blocks=[TextBlock(block_type='text', text='')]),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, additional_kwargs={'tool_call_id': 'call_Bntx2VFnOBcr0rd26XyG3kTI'}, blocks=[TextBlock(block_type='text', text='Agent shoe_retrieval_agent is now handling the request due to the following reason: User has provided specific shoe names for comparison..\nPlease continue with the current request.')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assista